In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    'udp_wcm_bronze_sap_bw',
    'sap_bw_wcm_vcdext01',
    proc_date,
    'temp_sap_vcdext01'
)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_sap_bw.sap_vcdext01
( 
hash_id BIGINT,
plant STRING,
calday DATE,
rpa_wid STRING,
rpa_tnr STRING,
_bic_zc_fgroup STRING,
_bic_zc_fname STRING,
recordmode STRING,
_bic_zc_fvalue STRING,
proc_date DATE,
file_creation_ts TIMESTAMP

)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'true',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'true'
)
""")

In [0]:
spark.sql(f"""
    CREATE OR REPLACE TEMP VIEW sap_vcdext01 AS
    SELECT
        FARM_FINGERPRINT(CONCAT(
            IFNULL(CAST(plant AS STRING), ""),
            IFNULL(CAST(calday AS STRING), ""),
            IFNULL(CAST(rpa_wid AS STRING), ""),
            IFNULL(CAST(rpa_tnr AS STRING), ""),
            IFNULL(CAST(_bic_zc_fgroup AS STRING), ""),
            IFNULL(CAST(_bic_zc_fname AS STRING), ""),
            IFNULL(CAST(recordmode AS STRING), ""),
            IFNULL(CAST(_bic_zc_fvalue AS STRING), "")
        )) hash_id,
        LTRIM('0', plant) AS plant,
        TO_DATE(calday,"yyyyMMdd") AS calday,
        LTRIM('0', rpa_wid) AS rpa_wid,
        LTRIM('0', rpa_tnr) AS rpa_tnr,
        _bic_zc_fgroup,
        _bic_zc_fname,
        recordmode,
        _bic_zc_fvalue,
        proc_date,
        file_creation_ts
    FROM (
        SELECT * EXCEPT(rn) FROM (
            SELECT *, ROW_NUMBER() OVER (PARTITION BY reqtsn, datapakid, record ORDER BY proc_date DESC) rn 
            FROM temp_sap_vcdext01
        ) a 
        WHERE rn = 1 
    ) a
""")


In [0]:
spark.sql(f"""
    MERGE INTO {catalog_name}.udp_wcm_silver_sap_bw.sap_vcdext01 main_tbl 
    USING sap_vcdext01 temp_tbl 
    ON main_tbl.hash_id = temp_tbl.hash_id
    WHEN NOT MATCHED THEN INSERT *
""")